In [1]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
path = "../tests/prompts.txt"
with open(path, "r") as f:
    texts = f.readlines()
    print(f"Loaded {len(texts)} prompts from {path}")
    

texts = texts[:40] 
    
messeges = [[{"role": "user", "content": tx}] for tx in texts]

Loaded 500 prompts from ../tests/prompts.txt


In [3]:
from src.vllm_service import VLLMService
from src.configs import LLMConfig, ServeConfig

from tqdm.auto import tqdm
from vllm import SamplingParams


llm_config = LLMConfig(
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    dtype="bfloat16",
)

serve_config = ServeConfig(
    gpu_ids=[1],
    host="127.0.0.1",
    port=None,
    startup_timeout=10 * 60,
    client_timeout=60,
    verbose=True,
)

sampling_params = SamplingParams(
    temperature=0.0,
    n=1,
    max_tokens=100,
)


service = VLLMService(llm_config, serve_config)

await service.start_async()

for i in tqdm(range(10)):

    chat_answers = await service.chat_async(messeges, sampling_params)
    gen_answers = await service.generate_async(texts, sampling_params)

await service.shutdown_async()

INFO 06-10 01:28:07 [__init__.py:243] Automatically detected platform cuda.
INFO 06-10 01:28:21 [__init__.py:243] Automatically detected platform cuda.
INFO 06-10 01:28:23 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-10 01:28:23 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-10 01:28:23 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-10 01:28:41 [config.py:793] This model supports multiple tasks: {'score', 'embed', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 06-10 01:28:41 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-10 01:28:44 [core.py:438] Waiting for init message from front-end.
INFO 06-10 01:28:44 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokeniz

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.15it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.15it/s]



INFO 06-10 01:28:50 [default_loader.py:280] Loading weights took 0.54 seconds
INFO 06-10 01:28:50 [gpu_model_runner.py:1549] Model loading took 2.3185 GiB and 2.682189 seconds
INFO 06-10 01:28:56 [backends.py:459] Using cache directory: /home/fre.gilad/.cache/vllm/torch_compile_cache/f7687da10c/rank_0_0 for vLLM's torch.compile
INFO 06-10 01:28:56 [backends.py:469] Dynamo bytecode transform time: 5.95 s
INFO 06-10 01:29:00 [backends.py:132] Directly load the compiled graph(s) for shape None from the cache, took 3.676 s
INFO 06-10 01:29:01 [monitor.py:33] torch.compile takes 5.95 s in total
INFO 06-10 01:29:02 [kv_cache_utils.py:637] GPU KV cache size: 1,269,440 tokens
INFO 06-10 01:29:02 [kv_cache_utils.py:640] Maximum concurrency for 131,072 tokens per request: 9.69x
INFO 06-10 01:29:17 [gpu_model_runner.py:1933] Graph capturing finished in 15 secs, took 0.86 GiB
INFO 06-10 01:29:17 [core.py:167] init engine (profile, create kv cache, warmup model) took 26.88 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

INFO 06-10 01:29:20 [chat_utils.py:419] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


In [4]:
# now use regular vllm.LLM to do the above

from vllm import LLM

llm = LLM(
    model=llm_config.model_name,
    dtype=llm_config.dtype,
)

for i in tqdm(range(10)):
    chat_answers = llm.chat(messeges, sampling_params, use_tqdm=False)
    gen_answers = llm.generate(texts, sampling_params, use_tqdm=False)


INFO 06-10 01:29:46 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-10 01:29:46 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-10 01:29:46 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-10 01:30:04 [config.py:793] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 06-10 01:30:04 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-10 01:30:07 [core.py:438] Waiting for init message from front-end.
INFO 06-10 01:30:07 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-10 01:30:14 [default_loader.py:280] Loading weights took 0.67 seconds
INFO 06-10 01:30:15 [gpu_model_runner.py:1549] Model loading took 2.3185 GiB and 3.999596 seconds
INFO 06-10 01:30:22 [backends.py:459] Using cache directory: /home/fre.gilad/.cache/vllm/torch_compile_cache/f7687da10c/rank_0_0 for vLLM's torch.compile
INFO 06-10 01:30:22 [backends.py:469] Dynamo bytecode transform time: 7.33 s
INFO 06-10 01:30:26 [backends.py:132] Directly load the compiled graph(s) for shape None from the cache, took 3.875 s
INFO 06-10 01:30:27 [monitor.py:33] torch.compile takes 7.33 s in total
INFO 06-10 01:30:28 [kv_cache_utils.py:637] GPU KV cache size: 1,269,440 tokens
INFO 06-10 01:30:28 [kv_cache_utils.py:640] Maximum concurrency for 131,072 tokens per request: 9.69x
INFO 06-10 01:30:45 [gpu_model_runner.py:1933] Graph capturing finished in 17 secs, took 0.86 GiB
INFO 06-10 01:30:45 [core.py:167] init engine (profile, create kv cache, warmup model) took 30.55 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

INFO 06-10 01:30:49 [chat_utils.py:419] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
